# Ruby Web Apps
### Owning Rails

 ## General notes on ruby
 - Everything in ruby is an object, which means you can call methods on anything (strings, fixnums, ints, etc.)
 - You can send methods dynamically using .send(:method_name)
 - You can write anything you want in the body of a class, including a method that applies to the class itself
   - To write a class method, prepend it with self.
 - `extend` adds all methods in the module as class level methods
 - `include` adds all methods in the module as instance level methods
 - Procs are objects like everything else and are used to wrap blocks so you can access them
   - Blocks are one thing that are NOT objects in ruby -- means you can't call methods on them
 - Lambdas check the arguments (and number of arguments) passed to them, procs don't
 - Lambdas return where `return` is called (i.e. exit the lambda), procs return from the enclosing method
 - `yield` keyword executes a block passed to a method

## Active Record
 - Each table in the database should correspond to a class in your app


## Taking advantage of Ruby's capabilities

- Ruby's powers make it an ideal language for meta programming and DSLs, like the one used by Rails' router:

In [8]:
class Router
  def initialize
    @routes = {}
  end
  
  def match(route)
    @routes.update route
  end
  
  def routes(&block)
    instance_eval(&block)
    p @routes
  end
end

Router.new.instance_eval { p @routes }

Router.new.routes do
  match '/users' => 'users#index'
  match '/login' => 'sessions#new'
end

{}
{"/users"=>"users#index", "/login"=>"sessions#new"}


{"/users"=>"users#index", "/login"=>"sessions#new"}

## Rack and the minimum required spec for a ruby app to run on a ruby web server

- Environment hash in, response out
- `.call` gets passed the request info as a hash
- The app has to return an array of params as the response to the web server, which gets parsed into a web page, including:
    - HTTP status code
    - Headers (hash)
    - Body
        - Must respond to `.each` as per rack specs, so we use an array in Ruby, because Ruby strings do not respond to `.each`
        
```ruby
class MyApp
  def call(env)
    [ 200,
      { 'Content-Type' => 'text/plain' },
      ['you requested ' + env['PATH_INFO']]
    ]
  end
end
```

## Minimum viable Ruby web server
- All Ruby web servers (unicorn, puma, etc.) perform 4 basic steps
- Difference between servers lies in how they process concurrent requests

1. Read data sent by browser
2. Parse data to a Hash
3. Call the rack app
4. Send the response to the browser with proper encoding

```ruby
class Server
  def process_request(socket)
    data = socket.read
    
    env = parse(data)
    
    status, headers, body = app.call(env)
    
    socket.write "HTTP/1.1 #{status} ..."
  end
end
```

- Rack-based servers use `config.ru` to start your app
- Has access to some special methods given by Rack (like `run`)

```ruby
require ::File.expand_path('../config/environment', __FILE__)
run Rails.application
```